# construct all stabilizers for a graph state

In [170]:
### loading some modules
import PauliStrings as ps

using DelimitedFiles
using Graphs

In [192]:
#=
    Utility functions
=#

### generate tyhe graph from the edge list
function generate_graph(data)
    data = data .+ 1
    le = length(data[1:end,1])
    elist = [(data[ii,1], data[ii,2]) for ii = 1:le]

    edges = Edge.(elist)
    g = SimpleGraph(edges)
    return g 
end

### construct the generators for a given graph
function get_generators(nqubits::Integer, G)
    all_generators = []
    
    for nn = 1:nqubits
        #-get neighbors
        neis = neighbors(G, nn)
        #-create auxiliary form of string
        temp = split(repeat("1",nqubits),"")
        temp[nn] = "X"
        #- filling the noidentity elements
        for ne in neis
            temp[ne] = "Z"
        end
        pstri = join(temp)

        #- construct the generator as a pauli string
        K = ps.Operator(nqubits)
        K += pstri
        #- stro generator
        push!(all_generators, K)
    end
    
    return all_generators
end

function generate_all_bitstrings(n::Int)
    if n < 0
        error("Length of bitstring cannot be negative.")
    end

    bitstrings = String[]
    # Iterate from 0 up to 2^n - 1
    for i in 0:(2^n - 1)
        # Convert the integer to its binary string representation
        # and pad with leading zeros to ensure a length of n
        push!(bitstrings, lpad(string(i, base=2), n, '0'))
    end
    return bitstrings
end

### build all the stabilizxers 
function get_all_stabilizers(nqubits::Integer, all_bs::Vector, generators::Vector)
    ### generate all stabilizers
    all_stabs = []
    all_coefs = []
    for ss in all_bs
        if ss == repeat("0", nqubits)
            push!(all_stabs, repeat("I", nqubits))
            push!(all_coefs, 1.0)
        else
            posis = findall(x -> x != '0', ss)
            S = prod(generators[kk] for kk in posis)
            coefs, Sstrings = ps.op_to_strings(S)

            stab = replace(Sstrings[1], "1" => "I")
            push!(all_stabs, stab)
            push!(all_coefs, coefs[1])
        end
    end
    return all_coefs, all_stabs
end

get_all_stabilizers (generic function with 2 methods)

In [193]:
### paths
pathg = "/Users/munm2002/Documents/projects/graph_states_for_ent_witness/all_graphs/"
patho = "/Users/munm2002/Documents/projects/graph_states_for_ent_witness/all_stabilizers_names/"

"/Users/munm2002/Documents/projects/graph_states_for_ent_witness/all_stabilizers_names/"

In [194]:
let
    ### system parameters
    N = 8

    ### load data
    data = readdlm(pathg*"graph_ring_cluster_$N"*"qubits.txt")[:,1:2]

    ### build graph
    gg = generate_graph(data)
    
    ### get all bit strings 
    all_bs = generate_all_bitstrings(N)
    
    ### generate the stabilizer generators
    generators = get_generators(N, gg)

    ### generate all stabilizers
    all_coefs, all_stabs = get_all_stabilizers(N, all_bs, generators)
    #real.(all_coefs)
end

256-element Vector{Float64}:
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
 -1.0
  1.0
  1.0
  1.0
 -1.0
  1.0
  ⋮
 -1.0
  1.0
 -1.0
 -1.0
 -1.0
 -1.0
  1.0
 -1.0
  1.0
 -1.0
 -1.0
  1.0

## ring graph state

In [196]:
let
    sizes = [4,5,6,7,8,9,10,11,12]

    for N in sizes
    
        ### load data
        data = readdlm(pathg*"graph_ring_cluster_$N"*"qubits.txt")[:,1:2]
    
        ### build graph
        gg = generate_graph(data)
        
        ### get all bit strings 
        all_bs = generate_all_bitstrings(N)
        
        ### generate the stabilizer generators
        generators = get_generators(N, gg)
    
        ### generate all stabilizers
        all_coefs, all_stabs = get_all_stabilizers(N, all_bs, generators)
        
        writedlm(patho*"names_ring_cluster_$N"*"qubits.txt", hcat(real.(all_coefs), all_stabs))
    end
end

## path graph state

In [203]:
@time let
    sizes = [4,5,6,7,8,9,10,11,12]

    for N in sizes
    
        ### load data
        data = readdlm(pathg*"graph_1D_cluster_$N"*"qubits.txt")[:,1:2]
    
        ### build graph
        gg = generate_graph(data)
        
        ### get all bit strings 
        all_bs = generate_all_bitstrings(N)
        
        ### generate the stabilizer generators
        generators = get_generators(N, gg)
    
        ### generate all stabilizers
        all_coefs, all_stabs = get_all_stabilizers(N, all_bs, generators)

        writedlm(patho*"names_1D_cluster_$N"*"qubits.txt", hcat(real.(all_coefs), all_stabs))
    end
end

  0.040926 seconds (706.79 k allocations: 55.008 MiB, 4.30% gc time)


## 2D cluster state

In [199]:
@time let
    sizes = [4,6,8,10,12]
    
    for ll = 1:length(sizes)
        N = sizes[ll]
        n = Int(N/2)
        
        ### load data
        data = readdlm(pathg*"graph_2D_cluster_$N"*"qubits_2x$n"*".txt")[:,1:2]
    
        ### build graph
        gg = generate_graph(data)
        
        ### get all bit strings 
        all_bs = generate_all_bitstrings(N)
        
        ### generate the stabilizer generators
        generators = get_generators(N, gg)
    
        ### generate all stabilizers
        all_coefs, all_stabs = get_all_stabilizers(N, all_bs, generators)
        
        writedlm(patho*"names_2D_cluster_$N"*"qubits_2x$n"*".txt", hcat(real.(all_coefs), all_stabs))
    end
end

  0.043876 seconds (483.97 k allocations: 37.708 MiB, 4.65% gc time)


In [200]:
@time let
    sizes = [9]
    
    for ll = 1:length(sizes)
        N = sizes[ll]
        n = Int(N/3)
        
        ### load data
        data = readdlm(pathg*"graph_2D_cluster_$N"*"qubits_3x$n"*".txt")[:,1:2]
    
        ### build graph
        gg = generate_graph(data)
        
        ### get all bit strings 
        all_bs = generate_all_bitstrings(N)
        
        ### generate the stabilizer generators
        generators = get_generators(N, gg)
    
        ### generate all stabilizers
        all_coefs, all_stabs = get_all_stabilizers(N, all_bs, generators)
        
        writedlm(patho*"names_2D_cluster_$N"*"qubits_3x$n"*".txt", hcat(real.(all_coefs), all_stabs))
    end
end

  0.003224 seconds (37.18 k allocations: 2.869 MiB)


## Tree graph state

In [201]:
@time let
    branches = [1,2,3]
    sizes = [4,7,10]
    
    for ll = 1:length(branches)
        n = branches[ll]
        N = sizes[ll]
        
        ### load data
        data = readdlm(pathg*"graph_tree_$n"*"branches_$N"*"qubits.txt")[:,1:2]
    
        ### build graph
        gg = generate_graph(data)
        
        ### get all bit strings 
        all_bs = generate_all_bitstrings(N)
        
        ### generate the stabilizer generators
        generators = get_generators(N, gg)
    
        ### generate all stabilizers
        all_coefs, all_stabs = get_all_stabilizers(N, all_bs, generators)
        
        writedlm(patho*"names_tree_$n"*"branches_$N"*"qubits.txt", hcat(real.(all_coefs), all_stabs))
    end
end

  0.007992 seconds (89.28 k allocations: 6.943 MiB, 25.28% gc time)
